In [9]:
!gdown 1GWMRVoiLa4UwZH4ibWRgasEbsfKXItbt
!unzip -q public_test.zip

Downloading...
From: https://drive.google.com/uc?id=1GWMRVoiLa4UwZH4ibWRgasEbsfKXItbt
To: /content/public_test.zip
100% 3.85G/3.85G [00:22<00:00, 171MB/s]


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!sudo apt install tesseract-ocr
!pip install -q pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 3s (1,811 kB/s)
debconf: unable to initi

In [4]:
!mkdir tessdata
!wget -O ./tessdata/eng.traineddata https://github.com/tesseract-ocr/tessdata_best/blob/main/eng.traineddata?raw=true

--2022-08-31 13:21:50--  https://github.com/tesseract-ocr/tessdata_best/blob/main/eng.traineddata?raw=true
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/tesseract-ocr/tessdata_best/raw/main/eng.traineddata [following]
--2022-08-31 13:21:50--  https://github.com/tesseract-ocr/tessdata_best/raw/main/eng.traineddata
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/main/eng.traineddata [following]
--2022-08-31 13:21:51--  https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/main/eng.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... conn

In [5]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 12122, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 12122 (delta 39), reused 52 (delta 27), pack-reused 12047
Receiving objects: 100% (12122/12122), 12.53 MiB | 611.00 KiB/s, done.
Resolving deltas: 100% (8330/8330), done.


In [6]:
!pip install -q -r /content/yolov5/requirements.txt

     |████████████████████████████████| 1.6 MB 40.2 MB/s 


Restart runtime after above step

In [1]:
import pandas as pd
import pytesseract
import re
import PIL
import difflib
import numpy as np
import random
import json
from glob import glob
from tqdm import tqdm
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

# OCR and matching

In [2]:
def is_similar(first, second, ratio):
    return difflib.SequenceMatcher(None, first, second).ratio() > ratio
    
def findid(text,pres_df):
    df = pd.DataFrame([text],columns =['text'])
    result = [s for f in df['text'] for s in pres_df['text'] if is_similar(f,s, 0.8)]
    df = pd.DataFrame(result,columns =['text'])['text'].unique()
    df = pd.DataFrame(df,columns =['text'])
    if not df.empty:
      # print(df.merge(pres_df,on = 'text' )['mapping'].values)
      return df.merge(pres_df,on = 'text' )['mapping'].unique().tolist() # Chỉnh khúc này nếu muốn dùng xác xuất
    else:
      return None

In [3]:
def remove_accents(input_str):
    s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
    s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
    s = ''
    for c in input_str:
        if c in s1:
            s += s0[s1.index(c)]
        else:
            s += c
    return s

def convert(txt:str):
    txt = txt.lower()
    txt = remove_accents(txt)
    txt = re.sub(r'^\d+\s*\)\s*', '', txt)
    txt = re.sub(r'\s*sl:.+', '', txt)
    return txt

# input: pres image path
# output: return list of pill name
def pres_process(pres_path:str):
    config_tesseract = "--tessdata-dir tessdata"

    txt=pytesseract.image_to_string(pres_path,lang='eng',config=config_tesseract)
    match = re.findall(r'^\d\s*\)\s*\w.+', txt,flags=re.MULTILINE|re.U)
    # remove 1) and SL: VIEN
    match = list(map(lambda x: convert(x), match))
    # convert to id
    ids = list(map(lambda x: findid(x,pres_df), match))
    ids = [item for sublist in ids for item in sublist]
    return match, ids

# Make pres_df

Có thể sử dụng file sẵn

In [4]:
!gdown 1T5FLTozjnfSCh4snMMG5qkrr-5AOIm1S

Downloading...
From: https://drive.google.com/uc?id=1T5FLTozjnfSCh4snMMG5qkrr-5AOIm1S
To: /content/pres_df.csv
100% 240k/240k [00:00<00:00, 96.6MB/s]


In [5]:
pres_df = pd.read_csv('pres_df.csv')

In [6]:
# files=glob('public_train\prescription\label\*.json')
# pres_df = pd.DataFrame()
# for file in tqdm(files):
#     temp_df = pd.read_json(file)
#     # add file name
#     temp_df['file_name'] = file.split('\\')[-1]
#     pres_df = pd.concat([pres_df,temp_df])
# pres_df = pres_df[pres_df['label']=='drugname']
# pres_df['mapping'] = pres_df['mapping'].astype(int)
# pres_df['text']=pres_df['text'].str.replace(r'^\d+\s*\)\s+','',regex=True).str.lower().apply(lambda x: remove_accents(x))
# pres_df.drop(['id'], axis=1, inplace=True)
# pres_df.head()
# pres_df.to_csv('pres_df.csv')


In [7]:
findid('droxicef 500mg 500mg',pres_df)

[19, 43]

# YOLO MODEL

In [8]:
import torch
yolo_model = torch.hub.load('ultralytics/yolov5','custom',path='/content/gdrive/MyDrive/AI4VN/VAIPE FINAL CODE/YOLO_MODELS/5s6_1280/weights/best.pt')

/usr/local/lib/python3.7/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untrusted repository. In a future release, this won't "
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-8-31 Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 280 layers, 12308200 parameters, 0 gradients, 1

# CLASS MODEL

In [9]:
import tensorflow as tf
image_width = 224
image_height = 224
class_model = tf.keras.models.load_model('/content/gdrive/MyDrive/AI4VN/VAIPE FINAL CODE/CLASS_MODELS/resnet50-new_yolo_bb-20220829-164052/model.h5')
class_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential (Sequential)        (None, 224, 224, 3)  0           ['input_3[0][0]']                
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 224, 224, 3)  0          ['sequential[0][0]']             
 ingOpLambda)                                                                                     
                                                                                            

In [10]:
def classification(img_path:str,ids)->pd.DataFrame:
    result = yolo_model(img_path,size=1280).pandas().xyxy[0]
    img = PIL.Image.open(img_path)
    img = PIL.ImageOps.exif_transpose(img)
    # img.show()
    for index,row in result.iterrows():
        # crop img
        x1,y1,x2,y2 = row['xmin'],row['ymin'],row['xmax'],row['ymax']
        crop = img.crop((x1,y1,x2,y2))
        # PIL image to tensorflow image
        crop = tf.convert_to_tensor(np.array(crop))
        crop = tf.image.resize(crop,(image_width,image_height))
        crop = tf.image.convert_image_dtype(crop,tf.float32)
        crop = crop.numpy()
        crop = crop.reshape((1,)+crop.shape)
        predict = class_model.predict([ids,crop],verbose=0)[0].argmax()
        result.loc[index,'class_id'] = predict
    return result

In [11]:
# l = np.array([0]*108)
# l[10]=1
# l[47]=1
# l[64]=1
# l[107]=1
# l = l.reshape((1,108))
# # from train data
# classification('VAIPE_P_0_0.jpg',l)
# # [
# #   { "x": 1520, "y": 1824, "h": 236, "w": 261, "label": 64 },
# #   { "x": 2227, "y": 2090, "h": 267, "w": 261, "label": 64 }
# # ]

# Run

In [12]:
# Load test data
path = '/content/public_test_new/pill_pres_map.json'
df = pd.read_json(path)
df['pres']=df['pres'].apply(lambda x:  '/content/public_test_new/prescription/image/'+x+'.png')
df['pill']=df['pill'].apply(lambda arr: list(map(lambda x: '/content/public_test_new/pill/image/'+x+'.jpg',arr)))
df.head()

,pres,pill
0,/content/public_test_new/prescription/image/VA...,[/content/public_test_new/pill/image/VAIPE_P_0...
1,/content/public_test_new/prescription/image/VA...,[/content/public_test_new/pill/image/VAIPE_P_1...
2,/content/public_test_new/prescription/image/VA...,[/content/public_test_new/pill/image/VAIPE_P_2...
3,/content/public_test_new/prescription/image/VA...,[/content/public_test_new/pill/image/VAIPE_P_3...
4,/content/public_test_new/prescription/image/VA...,[/content/public_test_new/pill/image/VAIPE_P_4...


In [13]:
predict = pd.DataFrame()
for index,row in tqdm(df.iterrows(),total=len(df)):
    pres_path = row['pres']
    pill_paths = row['pill']
    pill_names,ids=pres_process(pres_path)
    pill_ids = [0]*108
    pill_ids[107] = 1
    for id in ids:
      pill_ids[id] = 1
    pill_ids = np.array(pill_ids)
    pill_ids = pill_ids.reshape((1,)+pill_ids.shape)
    for pill_path in pill_paths:
      temp = classification(pill_path,pill_ids)
      temp['image_name'] = pill_path.split('/')[-1]
      predict = pd.concat([predict,temp])

 86%|████████▌ | 158/184 [20:16<03:20,  7.70s/it]


KeyboardInterrupt: ignored

In [ ]:
predict.head(20)

In [ ]:
predict.drop(['class','name'], axis=1, inplace=True)

In [ ]:
predict.rename(columns={'confidence':'confidence_score',
                        'xmin':'x_min',
                        'ymin':'y_min',
                        'xmax':'x_max',
                        'ymax':'y_max'
                        },inplace=True)

In [ ]:
predict = predict[['image_name', 'class_id', 'confidence_score', 'x_min', 'y_min', 'x_max', 'y_max']]
predict[['class_id','x_min','y_min','x_max','y_max']] = predict[['class_id','x_min','y_min','x_max','y_max']].astype(int)
predict['confidence_score'] = predict['confidence_score'].round(2)
predict.head()

In [ ]:
predict.to_csv('results.csv',index=False)

In [ ]:
predict=pd.read_csv('results.csv')
predict.head(50)

In [ ]:
from google.colab import files
files.download('results.csv') 
